In [ ]:
from copy import copy
from gymnasium.spaces import Discrete,MultiDiscrete
from pettingzoo import ParallelEnv

In [ ]:
gridheight=15
gridwidth=20

O player functions

In [ ]:
grid=[[0 for _ in range(gridwidth)] for _ in range(gridheight)]#6x7 grid board
def drop(col,symbol):#symbol is str
    i=5
    while i>=0:
        if ((grid[col][i-1]=="o") or (grid[col][i-1]=="x")):
            grid[col][i]=symbol
            break
        i-=1
def verticalchecko(x,y):
    for i in range(1,4):
        if grid[x][y-i]!="o":
            return 0
        else:
            return 1
def horizontalchecko(x,y):
    link=1
    for i in range(1,4):
        if ((x+i>=0) and (x+i<=6)):
            if grid[x+i][y]=="o":
                link+=1
            else:
                break
    for j in range(1,4):
        if ((x-j>=0) and (x-j<=6)):
            if grid[x-j][y]=="o":
                link+=1
            else:
                break
    if link>=4:
        return 1
    else: 
        return 0
def diagonalchecko(x,y):
    link=1
    for i in range(1,4):
        if ((x+i>=0) and (x+i<=6)):
            if grid[x+i][y+i]=="o":
                link+=1
            else:
                break
    for j in range(1,4):
        if ((x-j>=0) and (x-j<=6)):
            if grid[x-j][y-j]=="o":
                link+=1
            else:
                break
    if link>=4:
        return 1
    else: 
        return 0

Same functions for x player

In [ ]:
def verticalcheckx(x,y):
    for i in range(1,4):
        if grid[x][y-i]!="x":
            return 0
        else:
            return 1
def horizontalcheckx(x,y):
    link=1
    for i in range(1,4):
        if ((x+i>=0) and (x+i<=6)):
            if grid[x+i][y]=="x":
                link+=1
            else:
                break
    for j in range(1,4):
        if ((x-j>=0) and (x-j<=6)):
            if grid[x-j][y]=="x":
                link+=1
            else:
                break
    if link>=4:
        return 1
    else: 
        return 0
def diagonalcheckx(x,y):
    link=1
    for i in range(1,4):
        if ((x+i>=0) and (x+i<=6)):
            if grid[x+i][y+i]=="x":
                link+=1
            else:
                break
    for j in range(1,4):
        if ((x-j>=0) and (x-j<=6)):
            if grid[x-j][y-j]=="x":
                link+=1
            else:
                break
    if link>=4:
        return 1
    else: 
        return 0

In [ ]:
class CustomEnvironment(ParallelEnv): # creates custom environment
    metadata = {
        "name": "custom_environment_v0",
    }
    def __init__(self, maxsteps):
        # two agents, defining initial starting moves, points, action spaces and observation spaces
        self.column_o = None
        self.column_x = None
        self.timestep = None # starting timestep as 0
        self.maxsteps = maxsteps
        self.possible_agents = ["player_o", "player_x"]
        self.action_spaces = {
            a: Discrete(7) for a in self.possible_agents}
        self.observation_spaces = {
            #a: MultiDiscrete([len(MOVES), 20, 10]) for a in self.possible_agents,
                                   self.grid
                                   }
        self.grid = grid

    def reset(self, seed=None, options=None):
        self.agents=copy(self.possible_agents)
        self.timestep=0
        # when the game starts point is 0
        self.column_o=1
        self.column_x=1
        self.grid = [[0 for _ in range(7)] for _ in range(6)]
        #define observations
        observations={
            a:(self.column_o)
            for a in self.agents, self.grid
        }
        infos={a:{} for a in self.agents}
        return observations,infos

    def step(self, actions):
        if ((self.grid[col][5]="o")or(self.grid[col][5]="x")):
        #truncate
        #applying the move
        self.column_o = drop(actions["player_o"],"o")
        self.column_x = drop(actions["player_x"],"x")
        self.y_x
        rewards = {a: 0 for a in self.agents}
        #sets stopping value to 0
        terminations = {a: False for a in self.agents}
        truncations = {a: False for a in self.agents}
        i=5
        while i>=0:
            if ((grid[self.column_o][i-1]="o") or (grid[self.column_o][i-1]="x"))
                self.y_o=i
                break
            i-=1
        i=5
        while i>=0:
            if ((grid[self.column_x][i-1]="o") or (grid[self.column_x][i-1]="x"))
                self.y_x=i
                break
            i-=1
        #gives y of column_o and column_x
        if ((verticalchecko(self.column_o,self.y_o)) or (horizontalchecko(self.column_o,self.y_o)) or (diagonalchecko(self.column_o,self.y_o))):
            winner="player_o"
        elif ((verticalcheckx(self.column_x,self.y_x)) or (horizontalcheckx(self.column_x,self.y_x)) or (diagonalcheckx(self.column_x,self.y_x))):
            winner="player_x"
        else:
            winner=" "
        if winner == "player_o":
            rewards = {"player_o": 1, "player_x": -1}
            terminations = {a: True for a in self.agents}
            print(f"P_oWin")
        elif winner == "player_x":
            rewards = {"player_o": -1, "player_x": 1}
            terminations = {a: True for a in self.agents}
            print(f"P_xWin")
        else:
            truncations = {a: False for a in self.agents}
        self.timestep += 1

        if self.timestep > self.maxsteps:
            # stops the game if total steps goes over the maxstep count
            truncations = {"player_o": True, "player_x": True}

        observations = {
            #a: (self.moveselfo) for a in self.agents,
            self.grid
            }
        # info for debugging, default in most envs
        infos = {a: {"p_o_move": self.column_o, "p_x_move": self.column_x,
            "winner": winner if any(terminations.values()) else None
        } for a in self.agents
        }
        return observations, rewards, terminations, truncations, infos
    
    def render(self):
        print(f"{self.y_o,self.column_o}"
              f"{self.y_x,self.column_x}")

    def observation_space(self, agent):
        return self.observation_spaces[agent]

    def action_space(self, agent):
        return self.action_spaces[agent]

SyntaxError: invalid syntax (832779910.py, line 30)